In [8]:
# ! pip install spacy

In [9]:
import pandas as pd
import re
import logging
import spacy

In [10]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [11]:
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load(
#     "en_core_web_sm",
#     disable=["ner", "lemmatizer", "attribute_ruler"]
# )

In [13]:
data_dir = "data/"
dataframe_dir = data_dir + "dataframes/"
speeches_dir = data_dir + "speeches/"
speeches_pdf_dir = speeches_dir + "pdf/"
speeches_txt_dir = speeches_dir + "txt/"
speeches_extracted_dir = speeches_dir + "extracted/"
last_metadata_step = 'metadata_s03'
current_metadata_step = 'metadata_s04'

speech_suffix = "speeches.txt"
extracted_speech_suffix = lambda x,numb=True: f"speech_{x+1}.txt" if numb else "speech.txt"

In [14]:
metadata = pd.read_csv(f'{dataframe_dir}/{last_metadata_step}.csv')
metadata = metadata[metadata.text_based==True] # TODO change after image based pipeline is done

# Extract speech form txt
the regex to extract the speeches:

In [15]:
start_sg = re.compile(
    r"(?m)" # whole doc
    r"^\s*" # whitespace
    r"(?:\d+\.?\s*)?" # digit and string if existent e.g.(5. The Secretary...)
    # r"The\sSecretary-?General"
    r"(The\s+)?Secretary-General"
    r"(?:\s*\([^)]*\))?" # stuff in brackets if existent e.g.(spoke in spanish)
    r"[\s\n]*:" # :
    r"\s*", # whitespace if there
    re.IGNORECASE
)

next_speaker = re.compile(
    r"(?m)" # whole doc
    r"^\s*" # whitespace
    r"(?:\d+\.\s*)?" # digit and string if existent 
    r"(?:The|[A-Z]*[A-Za-z]*\.)" # "The" or "Mr."/"Mrs."/...
    r"(?:\s+[A-Z][A-Za-z\'\.\-]*){1,5}"  # 1 to 5 names
    r"(?:\s*\([^)]*\)){0,3}" # brackets, max 3 e.g.(Algeria) (spoke in spanish)
    r"\s*:" # :
    r"\s*", # whitespace if there)
)

### prepare speech metadata
The regex to count tokens, types and sentences

In [33]:
speech = """Thank you, Mr. President,
for giving me this opportunity to speak to the Assembly
about the financial situation of the Organization.
Many Member States have addressed this issue during
the general debate - often in candid terms.
It has been pointed out that there is no use talking
about reshaping the responsibilities of the United Nations
unless the resources are there to carry out those
responsibilities.
It has been said that the capacity of the United Nations
to respond to the demands made on it for peace-keeping
depends on the willingness of Member States to contribute
the resources needed to fulfil such a role.
And even more directly, it has been pointed out that
the United Nations works wonders with what Member
States do not pay it, but it simply cannot keep running on
exhausted credit.
At the end of August 1994, the Organization had debts
exceeding $1.7 billion. This amount includes about $1
billion owed to some 70 Member States for their troop and
equipment contributions to peace-keeping missions. At the
same time, the United Nations owes $400 million to
vendors and suppliers. It owes $325 million for budgetary
surpluses in prior years, amounts which should have been
returned to the Member States."""

In [ ]:
import numpy as np
def nlp_counting(text, nlp):
    doc = nlp(text)

    tokens = len(doc)
    types_ = len(set(t.text.lower() for t in doc))
    sentences = sum(1 for sentences in doc.sents)

    return tokens, types_, sentences

In [56]:
tokens, types, sentences = nlp_counting(speech, nlp)
print('------------')
tokens, types, sentences

------------


(253, 126, 9)

iterate each file and extract the regex:

In [ ]:
speech_metadata = []

def extract_speeches(df):
    # speech_metadata = []
    for row in df.itertuples():
        try:
            with open(f"{speeches_txt_dir+row.file_prefix}{speech_suffix}", "r", encoding="utf-8") as f:
                text = f.read()

            sg_matches = list(start_sg.finditer(text))
            multiple_matches = len(sg_matches) > 1

            for idx, sg_match in enumerate(sg_matches):
                start_idx = sg_match.end()
                stop_match = next_speaker.search(text, start_idx)
                end_idx = stop_match.start() if stop_match else len(text)
                speech = text[start_idx:end_idx].strip()

                with open(f"{speeches_extracted_dir+row.file_prefix}{extracted_speech_suffix(idx, multiple_matches)}", "w", encoding="utf-8") as file:
                    file.write(sg_match.group())
                    file.write("\n")
                    file.write(speech)
                exit
                # count tokens, types and sentences
                tokens, types, sentences = nlp_counting(speech, nlp)
                # append to new metadata for each speach individually
                speech_metadata.append({
                    "record_id": row.record_id,
                    "speaker": row.speaker,
                    "speaker_organization": row.speaker_organization,
                    "document_symbol": row.document_symbol,
                    "date": row.date,
                    "sg_number": row.sg_number,
                    "year": row.year,
                    "month": row.month,
                    "day": row.day,
                    "body": row.body,
                    "sub_body": row.sub_body,
                    "doc_number": row.doc_number,
                    "doc_type": row.doc_type,
                    "add_part": row.add_part,
                    "lang_field": row.lang_field,
                    "doc_url": row.doc_url,
                    "date_document": row.date_document,
                    "text_based": row.text_based,
                    "speech_index": idx,
                    "multiple_matches": multiple_matches,
                    "tokens": tokens,
                    "types": types,
                    "sentences": sentences
                })
            log.info(f"saved {len(sg_matches)} speech(es) for {row.document_symbol} where file prefix {row.file_prefix}")
        except Exception as e:
            log.error(f"while saving {row.document_symbol} where file prefix {row.file_prefix}")
            log.info(e)

    # speech_metadata = pd.DataFrame(speech_metadata)


''

In [32]:
extract_speeches(metadata)

Thank you, Mr. President,
for giving me this opportunity to speak to the Assembly
about the financial situation of the Organization.
Many Member States have addressed this issue during
the general debate - often in candid terms.
It has been pointed out that there is no use talking
about reshaping the responsibilities of the United Nations
unless the resources are there to carry out those
responsibilities.
It has been said that the capacity of the United Nations
to respond to the demands made on it for peace-keeping
depends on the willingness of Member States to contribute
the resources needed to fulfil such a role.
And even more directly, it has been pointed out that
the United Nations works wonders with what Member
States do not pay it, but it simply cannot keep running on
exhausted credit.
At the end of August 1994, the Organization had debts
exceeding $1.7 billion. This amount includes about $1
billion owed to some 70 Member States for their troop and
equipment contributions to peac

ERROR:__main__:while saving A/49/PV.28 where file prefix A_1994_49_PV.28_
INFO:__main__:object of type '_cython_3_2_1.generator' has no len()


To you, Sir, I offer my
congratulations on your appointment as Chairman of the
First Committee. With your diplomatic skills and expertise,
you are well qualified to guide this Committee through its
challenging agenda.
94-86696 (E) This record contains the
speeches delivered in th
only. They should be in A/C.1/49/PV.3
Official Records
. . . . . . . . . . . . . . . . (Ecuador)
I welcome this opportunity to address the First
Committee. I last addressed the Committee during the
resumed meetings of the forty-seventh session of the
General Assembly. That I come to address you again today
is an indication and an expression of the importance that I,
and the United Nations, attach to the Committee’s work.
The First Committee continues to make a major
contribution to international security and to the search for
a lasting global peace. The past year has confirmed the
wisdom of combining in the Committee the consideration of
arms control and disarmament with consideration of the
wider concerns of 

ERROR:__main__:while saving A/C.1/49/PV.3 where file prefix A_1994_C.1_49_PV.3_
INFO:__main__:object of type '_cython_3_2_1.generator' has no len()
ERROR:__main__:while saving A/49/PV.35 where file prefix A_1994_49_PV.35_
INFO:__main__:object of type '_cython_3_2_1.generator' has no len()


Five years ago, the General
Assembly decided that 1994 was to be the International
Year of the Family.
At the time, there was no consensus. Some did not
see the point of an International Year of the Family.
Opinions were divided as to what the Year was about.
Some people argued that support for the family
discriminates against those who prefer to live outside
family units. There were also disagreements over the
activities which should be organized to mark the year.
And so, in my speech last December to launch this
International Year, I urged that we concentrate on
practical outcomes: the Year should result in concrete
measures to strengthen the situation of the family around
the world.
Today I can report that a great deal of practical
action is indeed being taken, and more is planned. But I
can also report that in one key respect this International
Year of the Family has already exceeded expectations.
The International Year of the Family has stimulated
a worldwide debate. Many politica

...

ERROR:__main__:while saving A/56/PV.1 where file prefix A_2001_56_PV.1_
INFO:__main__:object of type '_cython_3_2_1.generator' has no len()
ERROR:__main__:while saving A/56/PV.7 where file prefix A_2001_56_PV.7_
INFO:__main__:object of type '_cython_3_2_1.generator' has no len()


Two weeks ago, as all
representatives will remember, we were looking
forward to this day, as the day when we would begin
our general debate. Many members expected to be
represented here by their head of State or Government
or by their foreign minister.
I myself had hoped to set out what I see as the
main priorities for our work for the next five years.
Alas, that was two weeks ago. Thirteen days ago, on a
day none of us is likely to forget, our host country and
our beloved host city were struck by a blow so
deliberate, heartless, malicious and destructive, that we
are all still struggling to grasp its enormity.
In truth, this was a blow, not against one city or
one country, but against all of us. It was an attack not
The need for a vigorous response to terrorism,
and for a sustained, comprehensive strategy to defeat it,
is not in doubt. But we also need to give greater
urgency to our humanitarian task of relieving the
victims of conflict and starvation, especially, at this
time, those 

ERROR:__main__:while saving A/56/PV.12 where file prefix A_2001_56_PV.12_
INFO:__main__:object of type '_cython_3_2_1.generator' has no len()
ERROR:__main__:while saving A/56/PV.24 where file prefix A_2001_56_PV.24_
INFO:__main__:object of type '_cython_3_2_1.generator' has no len()


The decision of the
Nobel Peace Prize Committee is truly an honour for the
whole of the United Nations, which means of course all
of its Member States and this Assembly, which
represents them. It is also an honour for all parts of the
Organization and, above all, for its dedicated staff
around the world. They work hard every day to make
the world a more just, a more peaceful and a happier
place. Many of them risk their lives, and they richly
deserve this award.
A year ago, your heads of State and Government
reaffirmed at the Millennium Summit the indispensable
role of the United Nations as a common house of the
entire human family. Now the Nobel Committee has
used this prize, in its own words,
“to proclaim that the only negotiable route to
global peace and cooperation goes by way of the
United Nations”.
In a world that is growing ever closer and more
interconnected, and yet is still torn by brutal conflicts
and cruel injustice, it is more important than ever that
humanity travel that r

KeyboardInterrupt: 

## look at speech metadata

In [ ]:
speech_metadata = pd.DataFrame(speech_metadata)

In [ ]:
speech_metadata.head(3)


""


In [ ]:
stop

NameError: name 'stop' is not defined

In [ ]:
speech_metadata.head(3)

,record_id,speaker,speaker_organization,document_symbol,date,sg_number,year,month,day,body,...,add_part,lang_field,doc_url,date_document,text_based,speech_index,multiple_matches,tokens,types,sentences
0,363235,"Boutros-Ghali, Boutros, 1922-2016",UN. Secretary-General,A/49/PV.28,1994-10-12,6,1994,10,12,A,...,NaN,English,https://digitallibrary.un.org/record/169004/fi...,1994-10-12,True,0,False,1215,453,72
1,365914,"Boutros-Ghali, Boutros, 1922-2016",UN. Secretary-General,A/C.1/49/PV.3,1994-10-17,6,1994,10,17,A,...,NaN,English,https://digitallibrary.un.org/record/170075/fi...,1994-10-17,True,0,False,2206,760,139
2,363570,"Boutros-Ghali, Boutros, 1922-2016",UN. Secretary-General,A/49/PV.35,1994-10-18,6,1994,10,18,A,...,NaN,English,https://digitallibrary.un.org/record/168316/fi...,1994-10-18,True,0,False,1241,463,77
